# core

> Utilites for building semantic knowledge graphs in rdflib using a fast.ai approach

In [ ]:
#| default_exp core

# RDFLib Parquet Storage

This module provides a fast, efficient way to store and retrieve RDF graphs using Parquet files. It wraps RDFLib's graph functionality with optimized Parquet serialization methods.

## Key Features

- Save RDF graphs to compressed Parquet files
- Load graphs from Parquet with optimized performance
- Preserve all RDF semantics (URIs, blank nodes, literals with datatypes)
- Memory-efficient batch processing for large graphs
- Fluent API for method chaining
- Simple integration with existing RDFLib code

## Use Cases

- Store large knowledge graphs efficiently
- Improve load/save times for RDF data
- Integrate semantic web data with data science pipelines
- Reduce storage requirements for RDF datasets
- Enable faster querying through columnar storage benefits

This module bridges the gap between semantic web technologies and modern data engineering practices, allowing RDF data to benefit from Parquet's columnar storage format advantages including compression, schema enforcement, and performance.

This is inspired by KGLabs implimentation [Performance analysis of serialization methods¶](https://derwen.ai/docs/kgl/ex2_0/#performance-analysis-of-serialization-methods) that showed parquet to be a reasonable triple store for local graphs. This also provides some convenience functions for constructing KGs.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import rdflib
import pandas as pd
import pyarrow as pa
from fastcore.all import *
from pathlib import Path
from typing import Union, List, Dict, Any, IO


In [ ]:
#| export
class KnowledgeGraph:
    "RDFLib wrapper with Parquet storage capabilities"
    _COL_NAMES = ["subject", "predicate", "object"]
    
    def __init__(self, g=None): 
        self.g = ifnone(g, rdflib.Graph())
    
    def __len__(self): return len(self.g)
    
    def __repr__(self): return f"KnowledgeGraph(triples={len(self)})"

    @delegates(pd.DataFrame.to_parquet)
    def save_parquet(self, path, compression="snappy", **kwargs):
        "Save RDF graph to Parquet file"
        rows = [{self._COL_NAMES[i]:o.n3() for i,o in enumerate(triple)} 
                for triple in self.g]
        df = pd.DataFrame(rows, columns=self._COL_NAMES)
        df.to_parquet(path, compression=compression, **kwargs)

    @delegates(pd.read_parquet)
    def load_parquet(self, path, batch_size=100000, **kwargs):
        "Load RDF graph from Parquet file with optimized performance"
        df = pd.read_parquet(path, **kwargs)
        total = len(df)
        
        # Process in batches to handle large graphs
        for start in range(0, total, batch_size):
            end = min(start + batch_size, total)
            batch = df.iloc[start:end]
            
            # Direct triple creation instead of parsing
            triples = []
            for _, row in batch.iterrows():
                s_str, p_str, o_str = row['subject'], row['predicate'], row['object']
                
                # Parse subject (URI or blank node)
                if s_str.startswith('<') and s_str.endswith('>'):
                    s = rdflib.URIRef(s_str[1:-1])
                elif s_str.startswith('_:'):
                    s = rdflib.BNode(s_str[2:])
                else:
                    s = rdflib.Literal(s_str)
                    
                # Parse predicate (always URI)
                if p_str.startswith('<') and p_str.endswith('>'):
                    p = rdflib.URIRef(p_str[1:-1])
                else:
                    p = rdflib.URIRef(p_str)
                    
                # Parse object (URI, blank node, or literal)
                if o_str.startswith('<') and o_str.endswith('>'):
                    o = rdflib.URIRef(o_str[1:-1])
                elif o_str.startswith('_:'):
                    o = rdflib.BNode(o_str[2:])
                elif o_str.startswith('"') or o_str.startswith("'"):
                    # This is a simplified approach - full N3 parsing is complex
                    # For production, consider using rdflib's parser directly
                    o = rdflib.Literal(o_str)
                else:
                    o = rdflib.Literal(o_str)
                    
                triples.append((s, p, o))
            
            # Add all triples in one batch
            self.g.addN((s, p, o, self.g) for s, p, o in triples)
        
        return self

    


In [ ]:
#| export
@patch
def add(self:KnowledgeGraph, triple):
    "Add a triple to the graph"
    self.g.add(triple)
    return self

In [ ]:
#| export 
@patch
def remove(self:KnowledgeGraph, triple):
    "Remove a triple from the graph"
    self.g.remove(triple)
    return self

In [ ]:
#| export 
@patch
def bind_ns(self:KnowledgeGraph, prefix, namespace):
    "Bind a namespace prefix"
    self.g.namespace_manager.bind(prefix, namespace)
    return self

In [ ]:
#| export 
@patch
def bind_namespaces(self:KnowledgeGraph, ns_dict):
    "Bind multiple namespace prefixes"
    for prefix, uri in ns_dict.items():
        ns = rdflib.Namespace(uri)
        self.g.namespace_manager.bind(prefix, ns)
    return self

In [ ]:
#| export
@patch
def query(self:KnowledgeGraph, q):
    "Run a SPARQL query"
    return self.g.query(q)

In [ ]:
#| export
@patch
def triples(self:KnowledgeGraph, pattern=None):
    "Return triples matching the pattern"
    pattern = ifnone(pattern, (None, None, None))
    return list(self.g.triples(pattern))

In [ ]:
#| export
@patch
def __getitem__(self:KnowledgeGraph, pattern):
    "Get triples matching a pattern using [] syntax"
    return list(self.g.triples(pattern))

@patch
def from_file(self:KnowledgeGraph, path, format=None):
    "Load graph from a file in any RDFLib-supported format"
    if format is None:
        format = Path(path).suffix.lstrip('.')
    self.g.parse(path, format=format)
    return self

@patch
def to_file(self:KnowledgeGraph, path, format=None):
    "Save graph to a file in any RDFLib-supported format"
    if format is None:
        format = Path(path).suffix.lstrip('.')
    self.g.serialize(destination=path, format=format)
    return self

@patch
def summary(self:KnowledgeGraph):
    "Print a summary of the graph"
    n_triples = len(self)
    n_subjects = len(set(s for s,_,_ in self.g))
    n_predicates = len(set(p for _,p,_ in self.g))
    n_objects = len(set(o for _,_,o in self.g))
    return dict(triples=n_triples, subjects=n_subjects, 
                predicates=n_predicates, objects=n_objects)


In [ ]:
#| hide
# Example: Working with DBpedia data about famous scientists
from urllib.request import urlopen

# Create our graph
kg = KnowledgeGraph()

# Load some real data from DBpedia (small subset about scientists)
scientist_data = """
@prefix dbr: <http://dbpedia.org/resource/> .
@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

dbr:Albert_Einstein a dbo:Scientist ;
    rdfs:label "Albert Einstein" ;
    dbo:field dbr:Physics ;
    dbo:birthDate "1879-03-14"^^<http://www.w3.org/2001/XMLSchema#date> ;
    dbo:deathDate "1955-04-18"^^<http://www.w3.org/2001/XMLSchema#date> .

dbr:Marie_Curie a dbo:Scientist ;
    rdfs:label "Marie Curie" ;
    dbo:field dbr:Chemistry, dbr:Physics ;
    dbo:birthDate "1867-11-07"^^<http://www.w3.org/2001/XMLSchema#date> ;
    dbo:deathDate "1934-07-04"^^<http://www.w3.org/2001/XMLSchema#date> .

dbr:Alan_Turing a dbo:Scientist ;
    rdfs:label "Alan Turing" ;
    dbo:field dbr:Computer_Science, dbr:Mathematics ;
    dbo:birthDate "1912-06-23"^^<http://www.w3.org/2001/XMLSchema#date> ;
    dbo:deathDate "1954-06-07"^^<http://www.w3.org/2001/XMLSchema#date> .
"""

# Parse the data
kg.g.parse(data=scientist_data, format="turtle")

# Show initial summary
print("Original graph:")
print(kg.summary())

Original graph:
{'triples': 17, 'subjects': 3, 'predicates': 5, 'objects': 14}


In [ ]:
#| hide
#| eval: false

# Test the optimized loader with timing
import time
from tqdm.auto import tqdm

# Generate a larger test graph with more diverse RDF terms
def create_test_graph(size=1000):
    kg = KnowledgeGraph()
    
    # Create some namespaces
    ex = rdflib.Namespace("http://example.org/")
    foaf = rdflib.Namespace("http://xmlns.com/foaf/0.1/")
    schema = rdflib.Namespace("http://schema.org/")
    
    # Bind namespaces
    kg.bind_ns("ex", ex)
    kg.bind_ns("foaf", foaf)
    kg.bind_ns("schema", schema)
    
    # Generate triples
    for i in tqdm(range(size)):
        # Subject (mix of URIs and blank nodes)
        if i % 10 == 0:
            s = rdflib.BNode(f"node{i}")
        else:
            s = ex[f"entity{i}"]
            
        # Add type
        kg.add((s, rdflib.RDF.type, ex.Resource))
        
        # Add string literal
        kg.add((s, schema.name, rdflib.Literal(f"Resource {i}")))
        
        # Add numeric literal
        kg.add((s, schema.position, rdflib.Literal(i)))
        
        # Add typed literal
        kg.add((s, schema.dateCreated, rdflib.Literal(f"2023-{(i%12)+1:02d}-{(i%28)+1:02d}", 
                                                    datatype=rdflib.XSD.date)))
        
        # Add language-tagged literal
        kg.add((s, schema.description, rdflib.Literal(f"Description of resource {i}", 
                                                     lang="en")))
        
        # Add relations to other resources
        kg.add((s, schema.related, ex[f"entity{(i+1)%size}"]))
        kg.add((s, schema.related, ex[f"entity{(i+size//2)%size}"]))
    
    return kg

In [ ]:
#| hide
#|eval: false

# Create and save a test graph
print("Creating test graph...")
test_size = 1000  # Adjust based on your system's capacity
start_time = time.time()
test_kg = create_test_graph(test_size)
create_time = time.time() - start_time
print(f"Created graph with {len(test_kg)} triples in {create_time:.2f} seconds")

Creating test graph...


NameError: name 'tqdm' is not defined

In [ ]:
#| hide
#| eval: false

# Save to Parquet
print("\nSaving to Parquet...")
start_time = time.time()
test_kg.save_parquet("test_graph.parquet")
save_time = time.time() - start_time
print(f"Saved to Parquet in {save_time:.2f} seconds")


Saving to Parquet...
Saved to Parquet in 0.02 seconds


In [ ]:
#| hide
#| eval: false

# Load with original method
print("\nLoading with original method...")
kg_orig = KnowledgeGraph()
# Replace the optimized method temporarily
orig_load_parquet = lambda self, path, **kwargs: (
    pd.read_parquet(path, **kwargs).apply(
        
        lambda row: self.g.parse(data=f"{row['subject']} {row['predicate']} {row['object']} .", format="ttl"),
        axis=1
    ) and self
)


Loading with original method...


In [ ]:
#| hide
#|eval: false

# Time the original method
start_time = time.time()
try:
    orig_load_parquet(kg_orig, "test_graph.parquet")
    orig_time = time.time() - start_time
    print(f"Loaded {len(kg_orig)} triples in {orig_time:.2f} seconds")
except Exception as e:
    print(f"Original method failed: {e}")
    orig_time = float('inf')

# Load with optimized method
print("\nLoading with optimized method...")
kg_opt = KnowledgeGraph()
start_time = time.time()
kg_opt.load_parquet("test_graph.parquet", batch_size=500)
opt_time = time.time() - start_time
print(f"Loaded {len(kg_opt)} triples in {opt_time:.2f} seconds")

# Compare results
print("\nComparison:")
print(f"Original method: {orig_time:.2f} seconds")
print(f"Optimized method: {opt_time:.2f} seconds")
if orig_time != float('inf'):
    print(f"Speedup: {orig_time/opt_time:.2f}x")

Original method failed: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

Loading with optimized method...
Loaded 7000 triples in 0.25 seconds

Comparison:
Original method: inf seconds
Optimized method: 0.25 seconds


In [ ]:
#| hide
#|eval: false

# Verify data integrity
print("\nVerifying data integrity...")
# Check triples count
if len(test_kg) == len(kg_opt):
    print(f"✓ Triple count matches: {len(test_kg)}")
else:
    print(f"✗ Triple count mismatch: original={len(test_kg)}, loaded={len(kg_opt)}")

# Run a few sample queries to verify correctness
q = """
SELECT (COUNT(*) as ?count) WHERE {
    ?s a ?type .
}
"""
orig_count = list(test_kg.query(q))[0][0].value
opt_count = list(kg_opt.query(q))[0][0].value
print(f"Type triples: original={orig_count}, loaded={opt_count}")

# Check literal handling
q = """
SELECT (COUNT(*) as ?count) WHERE {
    ?s <http://schema.org/dateCreated> ?date .
}
"""
orig_count = list(test_kg.query(q))[0][0].value
opt_count = list(kg_opt.query(q))[0][0].value
print(f"Date literals: original={orig_count}, loaded={opt_count}")


Verifying data integrity...
✓ Triple count matches: 7000
Type triples: original=1000, loaded=1000
Date literals: original=1000, loaded=1000


In [ ]:
#| hide
#|eval: false

# Benchmark: Parquet vs Turtle file storage
import time
import os

def benchmark_formats(graph_size=10000):
    print(f"Creating test graph with {graph_size} entities...")
    kg = create_test_graph(graph_size)
    triples = len(kg)
    print(f"Graph contains {triples} triples")
    
    # Benchmark saving
    formats = {
        "Parquet": {"save": lambda: kg.save_parquet("test_graph.parquet"), 
                   "load": lambda: KnowledgeGraph().load_parquet("test_graph.parquet")},
        "Turtle": {"save": lambda: kg.to_file("test_graph.turtle", format="turtle"), 
                  "load": lambda: KnowledgeGraph().from_file("test_graph.turtle", format="turtle")}
    }
    
    results = {}
    
    for fmt_name, actions in formats.items():
        # Test save
        print(f"\n--- {fmt_name} ---")
        start = time.time()
        actions["save"]()
        save_time = time.time() - start
        file_size = os.path.getsize(f"test_graph.{fmt_name.lower()[:7]}")
        print(f"Save time: {save_time:.2f}s")
        print(f"File size: {file_size/1024/1024:.2f} MB")
        
        # Test load
        start = time.time()
        loaded_kg = actions["load"]()
        load_time = time.time() - start
        print(f"Load time: {load_time:.2f}s")
        print(f"Loaded {len(loaded_kg)} triples")
        
        results[fmt_name] = {
            "save_time": save_time,
            "load_time": load_time,
            "file_size": file_size,
            "triples": len(loaded_kg)
        }
    
    # Calculate relative performance
    print("\n--- Performance Comparison ---")
    base_fmt = "Turtle"
    comp_fmt = "Parquet"
    
    save_speedup = results[base_fmt]["save_time"] / results[comp_fmt]["save_time"]
    load_speedup = results[base_fmt]["load_time"] / results[comp_fmt]["load_time"]
    size_reduction = results[base_fmt]["file_size"] / results[comp_fmt]["file_size"]
    
    print(f"Save speedup: {save_speedup:.2f}x faster with {comp_fmt}")
    print(f"Load speedup: {load_speedup:.2f}x faster with {comp_fmt}")
    print(f"Size reduction: {size_reduction:.2f}x smaller with {comp_fmt}")
    
    return results

# Run benchmark with different graph sizes
benchmark_formats(graph_size=5000)  # Adjust based on your system's capacity


Creating test graph with 5000 entities...


  0%|          | 0/5000 [00:00<?, ?it/s]

Graph contains 35000 triples

--- Parquet ---
Save time: 0.10s
File size: 0.30 MB
Load time: 0.95s
Loaded 35000 triples

--- Turtle ---
Save time: 0.45s
File size: 1.24 MB
Load time: 0.83s
Loaded 35000 triples

--- Performance Comparison ---
Save speedup: 4.58x faster with Parquet
Load speedup: 0.88x faster with Parquet
Size reduction: 4.13x smaller with Parquet


{'Parquet': {'save_time': 0.09809494018554688,
  'load_time': 0.9470059871673584,
  'file_size': 314087,
  'triples': 35000},
 'Turtle': {'save_time': 0.4496300220489502,
  'load_time': 0.8317821025848389,
  'file_size': 1298079,
  'triples': 35000}}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()